# Forward model for M/EEG source analysis

This notebook illustrates how to use features of the `stormdb.process`-module to:

* generate a BEM model based on pre-calculated tissue surfaces (_e.g._, inner skull, outer skull and outer skin)
* solve the BEM model (calculate interpolation coefficients for the surface elements based on node values)
* generate a _source space_ of possible dipole locations for a distributed (_e.g._, MNE) estimate
    * based on the cortical sheet (white-matter boundary) extracted using _Freesurfer_ `recon-all`
* generate a _forward operator_ for source modeling, using
    * solved BEM model
    * source space
    * head-to-mri transformation
    * an MEG measurement file containing the locations of the sensors relative to the head

_All computations are performed on the `hyades`-cluster._ The underlying code is based on the implementations found in `mne-python` and all outputs are written in the Neuromag FIF-format.

In [3]:
import os.path as op
from stormdb.process import MNEPython
from stormdb.base import mkdir_p  # mkdir -p

In [6]:
proj_name = 'MINDLAB2015_MEG-Sememene'
subject = '0010_HXN'  # full code must be given!
subjects_dir = op.join('/projects', proj_name, 'scratch', 'fs_subjects_dir')
fwd_dir = op.join('/projects', proj_name, 'scratch', 'fwd_operators')
#mkdir_p(fwd_dir)  # this must exist

bem_fname = op.join(subjects_dir, subject, 'bem', 'test-3L-sol.fif')
src_fname = op.join(subjects_dir, subject, 'bem', 'test-ico6-src.fif')

## Step 1a: BEM creation and solution

The BEM surfaces must be created first (see [this notebook](Generate BEM surfaces from MR.ipynb)). Depending on how many layers are to be used, _conductivity values_ must be specified for the desired number of layers. For example, for a 3-layer model, we might say:

In [7]:
conductivity = [0.3, 0.006, 0.3]  # brain, skull, skin
# conductivity = [0.3]  # for a single-layer (inner skull) model

In [8]:
mp = MNEPython(proj_name)

In [4]:
mp.prepare_bem_model(subject, bem_fname, subjects_dir=subjects_dir,
                     conductivity=conductivity)

## Step 1b: Setup source space

Take a `Freesurfer` surface (by default the white matter surface), downsample it to not more than 10,000 points per hemisphere, and save it to disk for later use. The `add_dist` can be used to allow later calls to `cortical patch statistics`.

__NB__ The BEM and source space calculations are independent, so we can use the same `MNEPython`-object to hold the jobs, and submit them together/in parallel.

In [9]:
mp.setup_source_space?

In [10]:
mp.setup_source_space(subject, src_fname)

OSError: Non-existent directory: /Users/cjb/UnixApps/freesurfer/subjects/0010_HXN

### Submit BEM and souce space together

In [8]:
mp.submit()

Cluster job submitted, job ID: 3829439
Cluster job submitted, job ID: 3829440


In [10]:
mp.status

#1 (3829439): Running on hyades04 (short.q)
#2 (3829440): Running on hyades06 (short.q)


## Step 2: Coregistration

See [this notebook](Head-to-MRI transformation for combined MEG and EEG.ipynb) for more details on this manual process.

## Step 3: Putting it all together

In [ ]:
mp = MNEPython(proj_name)
mp.make_forward_solution()